In [1]:
import gutenbergpy.textget
from gutenbergpy.gutenbergcache import GutenbergCache
import numpy as np
import pandas as pd

In [ ]:
GutenbergCache.create()
cache = GutenbergCache.get_cache()

Deleting old files

In [3]:
authors_list = cache.native_query("""
SELECT author.name
FROM books book
JOIN book_authors book_author ON book_author.bookid = book.id
JOIN authors author ON author.id = book_author.authorid
""").fetchall()
for author in set(np.array(authors_list).flatten()):
    if "Tolstoy" in author:
        print(author)

OperationalError: no such table: books

In [ ]:
books = cache.native_query("""
SELECT title.name, book.gutenbergbookid
FROM books book
JOIN book_authors book_author ON book_author.bookid = book.id
JOIN authors author ON author.id = book_author.authorid
JOIN titles title ON title.bookid = book.id
WHERE author.name = 'Tolstoy, Graf Leo'
""").fetchall()
books

In [ ]:
def safe_get_text_by_id(book_id):
    try:
        return gutenbergpy.textget.get_text_by_id(book_id)
    except Exception as e:
        return None

In [ ]:
texts = [(title, gutenbergpy.textget.strip_headers(text)) for title, book_id in books if (text := safe_get_text_by_id(book_id)) is not None]

In [ ]:
texts_df = pd.DataFrame(texts, columns=['title','text'])
texts_df

In [ ]:
texts_df["title"]